In [ ]:
import pandas as pd

api_id = your api id as number
api_hash = "your api hash"
phone = "your phone number"
session_name = "your session name"

In [ ]:
import socks
import re
import pandas as pd
from telethon import TelegramClient

# ============================================================
# Configuration Section
# ============================================================

# List of Telegram channel usernames to scrape proxy links from
channel_usernames = [
    "v2ray_dalghak", "Argo_VPN1", "vpn_shield", "configshere",
    "mrsoulb", "PrivateVPNs", "YamYamProxy", "DirectVPN", "V2rayNGX"
]

# Maximum number of messages to read from each channel
messages_limit = 300

# SOCKS5 proxy configuration (used to connect to Telegram)
# Typically required in restricted networks
proxy = (socks.SOCKS5, '127.0.0.1', 10808)

# ============================================================
# Regular Expression Patterns
# ============================================================

# Matches any standard HTTP or HTTPS URL
URL_REGEX = re.compile(r'https?://[^\s]+')

# Matches common proxy configuration links
# Supported formats: vmess, vless, trojan, ss, socks5
PROXY_REGEX = re.compile(
    r'\b(?:vmess|vless|trojan|ss|socks5)://[^\s]+',
    re.IGNORECASE
)

# ============================================================
# Asynchronous Scraping Function
# ============================================================

async def scrape_to_links():
    """
    Connects to Telegram using Telethon, scrapes messages from
    predefined channels, extracts URLs and proxy links, removes
    duplicates, and returns the results as a Pandas DataFrame.
    """

    # Create and open Telegram client session
    async with TelegramClient(
        'my_saved_session',
        api_id,
        api_hash,
        proxy=proxy
    ) as client:

        # Container for all extracted links
        all_links = []

        # Iterate through each configured Telegram channel
        for channel_username in channel_usernames:
            # Resolve channel entity
            channel = await client.get_entity(channel_username)

            # Iterate through recent messages in the channel
            async for msg in client.iter_messages(channel, limit=messages_limit):
                text = msg.text

                # Skip messages without text content
                if not text:
                    continue

                # Extract standard URLs from message text
                urls = URL_REGEX.findall(text)

                # Extract proxy configuration links
                proxy_links = PROXY_REGEX.findall(text)

                # Append extracted links to the main list
                all_links.extend(urls)
                all_links.extend(proxy_links)

        # Remove duplicate links while preserving original order
        unique_links = list(dict.fromkeys(all_links))

        # Convert results to a Pandas DataFrame
        df_links = pd.DataFrame({"links": unique_links})

        return df_links

# ============================================================
# Execution (Jupyter Notebook / Async Context)
# ============================================================

# Run the asynchronous scraping function
df_links = await scrape_to_links()

# Display the resulting DataFrame
df_links


,links
0,ss://Y2hhY2hhMjAtaWV0Zi1wb2x5MTMwNTpnMFVnZnNkW...
1,vless://570eb4f2-3563-4ea1-9291-21efda3c01bb@i...
2,vless://6fe32852-5f46-4090-8306-f5b419d6a469@9...
3,ss://Y2hhY2hhMjAtaWV0Zi1wb2x5MTMwNTp5UmR4dkszM...
4,ss://Y2hhY2hhMjAtaWV0Zi1wb2x5MTMwNTpvWklvQTY5U...
...,...
4946,vless://b1335e20-6006-4bc1-a791-8820772903fd@6...
4947,vless://53fa8faf-ba4b-4322-9c69-a3e5b1555049@1...
4948,vless://7e5813d0-e5ac-437c-ba4e-2f8c17a84aae@1...
4949,vless://7e5813d0-e5ac-437c-ba4e-2f8c17a84aae@1...


In [ ]:
import requests

# ============================================================
# Configuration
# ============================================================

# URL of a public repository providing aggregated proxy configurations
url = "https://raw.githubusercontent.com/Epodonios/v2ray-configs/refs/heads/main/All_Configs_Sub.txt"

# SOCKS5 proxy configuration used for fetching the file
# This allows access in restricted or filtered networks
proxies = {
    "http":  "socks5h://127.0.0.1:10808",
    "https": "socks5h://127.0.0.1:10808"
}

# ============================================================
# Fetch Remote Configuration File
# ============================================================

# Send HTTP GET request through the SOCKS5 proxy
response = requests.get(url, proxies=proxies, timeout=30)

if response.status_code == 200:
    # Read file content as plain text
    content = response.text

    # Split content into lines and remove empty entries
    urls2 = [line.strip() for line in content.splitlines() if line.strip()]

    print(f"Found {len(urls2)} URLs")
else:
    # Handle unsuccessful request
    print("Failed to fetch file:", response.status_code)

# ============================================================
# Data Processing and Filtering
# ============================================================

# Convert fetched links into a DataFrame
vless_df = pd.DataFrame({'links': urls2})

# Merge newly fetched links with previously scraped links
vless_df = pd.concat([df_links, vless_df], ignore_index=True)

# Keep only VMess configurations
vless_df = vless_df[vless_df['links'].str.startswith(('vmess'))]

# Remove empty values
vless_df = vless_df.dropna()

# Remove duplicate configurations
vless_df = vless_df.drop_duplicates(subset="links").reset_index(drop=True)

# ============================================================
# Save Results
# ============================================================

# Export final cleaned VMess configurations to Excel
vless_df.to_excel(
    '/home/alireza/github/My-automated-life/Config_Automation/data/1)Scrapped_VMESS_Configs.xlsx'
)

# Display resulting DataFrame
vless_df


Found 6481 URLs


,links
0,vmess://eyJhZGQiOiAiOTUuMjE2LjE4My4xMTMiLCAiYW...
1,vmess://eyJhZGQiOiAidHIxLXNtYXJ0LmFkZWxwaW5nLm...
2,vmess://eyJhZGQiOiAiMTQwLjI0OC4xMzcuOTkiLCAiYW...
3,vmess://eyJhZGQiOiAiQ3Bhbm5lbC5pcmFudWNzaG9wLm...
4,vmess://eyJ2IjogIjIiLCAicHMiOiAidjJyYXlfZGFsZ2...
...,...
792,vmess://ewogICAgImFkZCI6ICIxODAuMjE1LjEzMC4xMj...
793,vmess://ewogICAgImFkZCI6ICIxMzQuMTk1LjE5OC4xND...
794,vmess://ewogICAgImFkZCI6ICJoZ3Ryb2phbi56YWJjLm...
795,vmess://ewogICAgImFkZCI6ICIxMzQuMTk1LjE5OC4xND...


In [ ]:
import base64
import json

# ============================================================
# VMess URL → Xray Configuration Converter
# ============================================================

def vmess_url_to_xray_config(vmess_url: str) -> dict:
    """
    Converts a VMess URL into a valid Xray JSON configuration.

    Parameters:
        vmess_url (str): VMess-formatted proxy URL

    Returns:
        dict: Xray-compatible configuration dictionary
              Returns None if parsing or decoding fails
    """
    try:
        # Ensure the provided link is a VMess URL
        if not vmess_url.startswith("vmess://"):
            raise ValueError("Not a vmess URL")

        # Remove the scheme prefix
        encoded = vmess_url[len("vmess://"):]

        # Fix missing Base64 padding if required
        padding = '=' * (-len(encoded) % 4)
        decoded_bytes = base64.b64decode(encoded + padding)

        # Decode VMess JSON payload
        vmess_data = json.loads(decoded_bytes.decode("utf-8"))

        # ====================================================
        # Base Xray Configuration Template
        # ====================================================

        config = {
            "log": {
                "loglevel": "warning"
            },
            "inbounds": [
                {
                    "port": 10808,
                    "listen": "127.0.0.1",
                    "protocol": "socks",
                    "settings": {
                        "udp": True
                    }
                }
            ],
            "outbounds": [
                {
                    "protocol": "vmess",
                    "settings": {
                        "vnext": [
                            {
                                "address": vmess_data["add"],
                                "port": int(vmess_data["port"]),
                                "users": [
                                    {
                                        "id": vmess_data["id"],
                                        "alterId": int(vmess_data.get("aid", 0)),
                                        "security": vmess_data.get("scy", "auto")
                                    }
                                ]
                            }
                        ]
                    },
                    "streamSettings": {
                        "network": vmess_data.get("net", "tcp")
                    }
                }
            ]
        }

        # ====================================================
        # TCP Transport Settings
        # ====================================================

        if vmess_data.get("net") == "tcp":
            config["outbounds"][0]["streamSettings"]["tcpSettings"] = {
                "header": {
                    "type": vmess_data.get("type", "none")
                }
            }

        # ====================================================
        # TLS Security Settings
        # ====================================================

        if vmess_data.get("tls") == "tls":
            config["outbounds"][0]["streamSettings"]["security"] = "tls"
            config["outbounds"][0]["streamSettings"]["tlsSettings"] = {
                "serverName": vmess_data.get("sni", ""),
                "allowInsecure": vmess_data.get("insecure", "0") == "1"
            }

        return config

    except:
        # Any decoding or parsing failure results in exclusion
        return None


# ============================================================
# Apply Conversion to DataFrame
# ============================================================

# Convert VMess URLs into Xray JSON configs
vless_df['json_config'] = vless_df['links'].apply(vmess_url_to_xray_config)

# Remove rows where conversion failed
vless_df = vless_df.dropna(subset=['json_config'])

# Normalize boolean and quotation formatting for JSON compatibility
vless_df["json_config"] = vless_df["json_config"].apply(
    lambda x: str(x).replace("'", '"')
                     .replace("True", "true")
                     .replace("False", "false")
    if pd.notnull(x) else x
)

# ============================================================
# Save Output
# ============================================================

# Export final Xray configurations to Excel
vless_df.to_excel(
    '/home/alireza/github/My-automated-life/Config_Automation/data/2)Scrapped_VMESS_Configs_json.xlsx'
)

# Display resulting DataFrame
vless_df

/tmp/ipykernel_67946/2453500888.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vless_df["json_config"] = vless_df["json_config"].apply(


,links,json_config
0,vmess://eyJhZGQiOiAiOTUuMjE2LjE4My4xMTMiLCAiYW...,"{""log"": {""loglevel"": ""warning""}, ""inbounds"": [..."
1,vmess://eyJhZGQiOiAidHIxLXNtYXJ0LmFkZWxwaW5nLm...,"{""log"": {""loglevel"": ""warning""}, ""inbounds"": [..."
2,vmess://eyJhZGQiOiAiMTQwLjI0OC4xMzcuOTkiLCAiYW...,"{""log"": {""loglevel"": ""warning""}, ""inbounds"": [..."
3,vmess://eyJhZGQiOiAiQ3Bhbm5lbC5pcmFudWNzaG9wLm...,"{""log"": {""loglevel"": ""warning""}, ""inbounds"": [..."
4,vmess://eyJ2IjogIjIiLCAicHMiOiAidjJyYXlfZGFsZ2...,"{""log"": {""loglevel"": ""warning""}, ""inbounds"": [..."
...,...,...
792,vmess://ewogICAgImFkZCI6ICIxODAuMjE1LjEzMC4xMj...,"{""log"": {""loglevel"": ""warning""}, ""inbounds"": [..."
793,vmess://ewogICAgImFkZCI6ICIxMzQuMTk1LjE5OC4xND...,"{""log"": {""loglevel"": ""warning""}, ""inbounds"": [..."
794,vmess://ewogICAgImFkZCI6ICJoZ3Ryb2phbi56YWJjLm...,"{""log"": {""loglevel"": ""warning""}, ""inbounds"": [..."
795,vmess://ewogICAgImFkZCI6ICIxMzQuMTk1LjE5OC4xND...,"{""log"": {""loglevel"": ""warning""}, ""inbounds"": [..."
